In [132]:
import numpy as np
import random
import pandas as pd 
from copy import deepcopy

In [133]:
def rastrigin(x):
    A = 10
    n = len(x)
    return -(A*n + sum(x[i]**2 - A*np.cos(2*np.pi*x[i]) for i in range(n)))

In [134]:
def rosenbrock(x):
    a = 1
    b = 100
    return -((a - x[0])**2 + b*(x[1] - x[0]**2)**2)


In [135]:
class Individual:
    global_best_position=None
    global_best_fitness=None
    def __init__(self,bounds,velocity_calc,calc_fitness,age=0,life_expectancy=120):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])        

        self.life_expectancy=life_expectancy
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))
        self.calc_fitness=calc_fitness
        self.fitness=calc_fitness(self.position)
        self.age=age

        self.velocity_calc=velocity_calc
        
        self.personal_best_fitness=self.fitness
        self.personal_best_position=self.position.copy()

        self.patriarch_best_fitness=self.fitness
        self.patriarch_best_position=self.position.copy()
        
        if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    
        
    def update_position(self):
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)
        self.update_fitness()
       
                
    def update_velocity(self):
        cognitive_velocity = self.velocity_calc["cognitive"](self.age,self.life_expectancy)* (self.personal_best_position - self.position)
        social_velocity = self.velocity_calc["group"](self.age,self.life_expectancy)* (Individual.global_best_position - self.position)
        inertia = self.velocity_calc["inertia"](self.age,self.life_expectancy) * self.velocity
        patriarch_velocity= self.velocity_calc["patriarch"](self.age,self.life_expectancy)*(self.patriarch_best_position-self.position)
        self.velocity = inertia + cognitive_velocity + social_velocity
    def update_fitness(self):
        self.fitness=self.calc_fitness(self.position)
        if self.personal_best_fitness is None or self.fitness>self.personal_best_fitness:
            self.personal_best_fitness=self.fitness
            self.personal_best_position=self.position.copy()
            if self.patriarch_best_fitness is None or self.patriarch_best_fitness<self.fitness:
                self.patriarch_best_fitness=self.fitness
                self.patriarch_best_position=self.position.copy()
            
                if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                    Individual.global_best_fitness=self.fitness
                    Individual.global_best_position=self.position.copy()
    def mutate(self,mut_prob):
        #TODO
        for i in range(len(self.position)):
            if random.random()<mut_prob:
                self.position[i]+=0.01*random.uniform(self.lower_bounds[i],self.upper_bounds[i])
            if random.random()<mut_prob:
                self.velocity[i]+=0.01*random.uniform(self.lower_bounds[i],self.upper_bounds[i])
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)

In [136]:
def crossover(p1:Individual,p2:Individual,c1:Individual,c2:Individual):
    coef=random.random()
    c1.position=coef*p1.position + (1-coef)*p2.position
    c1.velocity=coef*p1.velocity + (1-coef)*p2.velocity

    c2.position=coef*p2.position +(1-coef)*p1.position
    c2.velocity= coef*p2.velocity +(1-coef)*p1.velocity

    best_patriarch_relative=max([p1,p2,c1,c2],key=lambda x:x.patriarch_best_fitness)
    c1.patriarch_best_fitness=best_patriarch_relative.patriarch_best_fitness
    c1.patriarch_best_position=best_patriarch_relative.patriarch_best_position.copy()

    c2.patriarch_best_position=best_patriarch_relative.patriarch_best_position.copy()
    c2.patriarch_best_fitness=best_patriarch_relative.patriarch_best_fitness
    
    c1.age=0
    c2.age=0

In [137]:
def mutation(ind:Individual,mut_prob=0.01):
    #ind.mutate(mut_prob)
    pass

In [138]:
def selection(population,tournament_size):
    allowed=random.sample(population,tournament_size)
    return max(allowed,key=lambda x:x.fitness)

In [209]:
velocity_calculator={
    "cognitive": lambda age,life_expectancy:min(age,90/120*life_expectancy)/life_expectancy ,
    "group" : lambda age ,life_expectancy:min(max(90/120*life_expectancy-age,0),90/120*life_expectancy)/life_expectancy,
    "inertia" : lambda age , life_expectancy:(120/120),
    "patriarch" :lambda age,life_expectancy:(100/120)
    
}
vezbe_velocity_calc={
    "cognitive": lambda age,life_expectancy:0.7 ,
    "group" : lambda age ,life_expectancy:1.0,
    "inertia" : lambda age , life_expectancy:(120/120),
    "patriarch" :lambda age,life_expectancy:(60/120)
    
}

def psogahybrid(pop_size, num_dimensions, num_iters,velocity_calc,fitness_calc,life_expectancy=120,tournament_size=4):
    bounds = [(-5.12, 5.12) for _ in range(num_dimensions)]
    
    population = [Individual(bounds=bounds,
                      calc_fitness=fitness_calc,life_expectancy=life_expectancy,velocity_calc=velocity_calc,age=i%life_expectancy
                      ) for i in range(pop_size)]
    print(Individual.global_best_fitness)
    
                
    for _ in range(num_iters):
        for i in range(len(population)):
            
            population[i].update_velocity()
            population[i].update_position()
            population[i].age+=1
            population[i].update_fitness()
            if population[i].age==life_expectancy:
                
                toPairWith=selection(population,tournament_size)
                child1=Individual(bounds=bounds,calc_fitness=fitness_calc,velocity_calc=velocity_calc,life_expectancy=life_expectancy)
                child2=Individual(bounds=bounds,calc_fitness=fitness_calc,velocity_calc=velocity_calc,life_expectancy=life_expectancy)
                crossover(population[i],toPairWith,child1,child2)
                mutation(child1)
                mutation(child2)
                population[i]=max([child1,child2],key=lambda x:x.fitness)
                population[i].update_fitness()

            
    
    print(Individual.global_best_position)
    print(Individual.global_best_fitness)
    
    Individual.global_best_position = None
    Individual.global_best_fitness = None

In [222]:
psogahybrid(pop_size=50, num_dimensions=2, num_iters=100,velocity_calc=vezbe_velocity_calc,life_expectancy=10,fitness_calc=rosenbrock)

-2.643290236837618
[1.00004594 1.00007239]
-4.0146988732045146e-08


In [141]:
#cultural generational learning

Ideja algoritma:
   Nalik goal driven generational learning-u samo sto imamo i patriarch_best_position
   kada update-ujemo velocity.To je jedinka sa najboljim fitnesom u njegovom porodicnom stablu.
   Pustamo da i ona utice na ciljeve kretanja jedinke iz "kulturoloskih razloga" ako ima smisla.



In [2]:
maxtime=10**(-2)
num_iters=1000

In [ ]:
sum=0
for i in range(num_iters):
    sum+=psogahybrid(pop_size=50,num_dimensions=2,ve,num_iters=10000)
average=sum/num_iters
print(average)